In [1]:
import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
# from tensorflow.keras.utils import np_utils
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras import layers 
import tensorflow.keras.backend as K 

from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler


import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
# import plotly.express as px
# import plotly.graph_objects as go

from math import sqrt

In [2]:
lstm_train_X = pd.read_csv("ERA\\ERA_train_x.csv")
lstm_train_y = pd.read_csv("ERA\\ERA_train_y.csv")

lstm_test_X = pd.read_csv("ERA\\ERA_test_x.csv")
lstm_test_y = pd.read_csv("ERA\\ERA_test_y.csv")

In [3]:
team = list(lstm_train_X.T_ID.unique())
year = list(lstm_train_y.YEAR.unique())

In [4]:
team, year

(['LG', 'HH', 'NC', 'HT', 'SK', 'KT', 'WO', 'LT', 'SS', 'OB'],
 [2016, 2017, 2018, 2019])

In [5]:
model_dict = dict()
hist_dict = dict()
test_pred_df = pd.DataFrame([],columns = ['YEAR','T_ID','y','y_pred','MSE','MSE_avg'])

idx = 0
for y in year:
    tmp1 = lstm_train_X[lstm_train_X["YEAR"] == y]
    tmp2 = lstm_train_y[lstm_train_y["YEAR"] == y]
    tmp3 = lstm_test_X[lstm_test_X["YEAR"] == y]
    tmp4 = lstm_test_y[lstm_test_y["YEAR"] == y]
    for t in team:
        name = '{}{}'.format(y,t)
        print(name,"=======================================")
        
        X_train = tmp1[tmp1["T_ID"] == t].drop(["T_ID","YEAR"],axis = 1)
        y_train = tmp2[tmp2["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
        X_test = tmp3[tmp3["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
        y_test = tmp4[tmp4["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
        
        X_train_v = X_train.values
        y_train_v = y_train.values

        X_test_v = X_test.values
        y_test_v = y_test.values
        """
        X_train_t = X_train_v.reshape(X_train_v.shape[0], 2,X_train_v.shape[1]//2)
        X_test_t = X_test_v.reshape(X_test_v.shape[0], 2,X_test_v.shape[1]//2)
        """
        
        ## model
        K.clear_session() 
        
        model = Sequential()
        optimizer = Adam(lr=0.01)
        model.add(Dense(1024, input_shape = [X_train.shape[1]]))
        model.add(Dense(1024))
        model.add(Dense(1024))
        model.add(Dense(1024))
        model.add(Dense(1024))
        model.add(Dense(1)) # output = 1
        model.compile(loss='mean_squared_error', optimizer=optimizer,metrics=['mae'])

       
        model.summary()
        
        early_stop = EarlyStopping(monitor='loss', mode = 'min',patience=2, verbose=1)

        hist1 = model.fit(X_train, y_train, epochs=100,
                  batch_size=1, verbose=1, callbacks=[early_stop])
        ##
        
        model_dict[name] = model
        hist_dict[name] = hist1
        
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        
        ######### MSE AVG#########
        mse_avg = mean_squared_error(y_test,[y_train.mean()[0]])
        
        
        test_pred_df.loc[idx,:] = [y,t,y_test_v.reshape(-1)[0],y_pred.reshape(-1)[0], mse,mse_avg]

        idx += 1

test_pred_df[['y','y_pred',"shift_AVG_1","shift_AVG_2",'MSE','MSE_avg']] = test_pred_df[['y','y_pred',"shift_AVG_1","shift_AVG_2",'MSE','MSE_avg']].astype(float)

2016LG =======================================


W0927 20:11:53.934772 22168 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              47104     
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 1025      
Total params: 4,246,529
Trainable params: 4,246,529
Non-trainable params: 0
______________________________________________

W0927 20:12:19.665949 22168 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


Epoch 00016: early stopping
2016HH =======================================


...

W0927 20:29:07.784954 22168 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              47104     
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 1025      
Total params: 4,246,529
Trainable params: 4,246,529
Non-trainable params: 0
______________________________________________

W0927 20:29:50.234410 22168 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


Epoch 00028: early stopping


KeyError: "['shift_AVG_2', 'shift_AVG_1'] not in index"

In [6]:
test_pred_df

,YEAR,T_ID,y,y_pred,MSE,MSE_avg
0,2016,LG,3.64977,-16.5181,406.742,2.25378
1,2016,HH,5.25811,-17.8896,535.816,0.0274959
2,2016,NC,3.57534,-14.2021,316.036,2.08141
3,2016,HT,4.12083,380.035,141312,1.58214
4,2016,SK,5.14488,-87.4744,8578.34,0.0145698
5,2016,KT,6.31452,59.1235,2788.79,0.0232326
6,2016,WO,5.70484,-7.37659,171.124,0.343514
7,2016,LT,5.84929,164.623,25209.1,0.00463758
8,2016,SS,5.4,-4.99729,108.104,0.103031
9,2016,OB,4.71028,3.22168,2.21594,0.0133325
